In [19]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [20]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [21]:
tf.keras.backend.clear_session()

In [22]:
subject = 'Pará - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 - 30

In [23]:
data = pd.read_csv('2003_mo_model_input_PA.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Pará - Desemprego,Pará - Produção de Cimento (t),Pará - value,Pará - IDH,Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado,Pará - Consumo de Cimento (t)
0,2003-1,0.724032,11.520143,1.639718,1.036534,1.772069e+08,7.330309e+06,0.969649,8.348543,49.618560,0.312867,0.669564,7.776091e+07,4.250430e+06,9.220374,6.851269e+07,84.701
1,2003-2,0.690297,11.189862,1.378899,0.993449,1.773884e+08,7.335910e+06,0.950783,8.342554,49.658098,0.312657,0.669764,7.787181e+07,4.255476e+06,9.225493,6.856365e+07,80.221
2,2003-3,0.669681,10.820792,1.924317,0.973020,1.775699e+08,7.341511e+06,0.938332,8.336566,49.906876,0.311736,0.669963,7.798271e+07,4.260521e+06,9.230612,6.861461e+07,76.122
3,2003-4,0.660494,10.417840,1.331174,0.940489,1.777514e+08,7.347112e+06,0.926401,8.330578,49.864305,0.310493,0.670163,7.809361e+07,4.265566e+06,9.235731,6.866557e+07,76.544
4,2003-5,0.648337,9.959690,1.736072,0.917493,1.779329e+08,7.352713e+06,0.951683,8.324589,50.179753,0.309128,0.670362,7.820451e+07,4.270611e+06,9.240851,6.871652e+07,83.445
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.899018,0.710173,NaN,NaN,NaN,NaN,NaN,156.442
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.416567,0.708384,NaN,NaN,NaN,NaN,NaN,145.777
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.559167,0.706038,NaN,NaN,NaN,NaN,NaN,139.233
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.696005,0.702147,NaN,NaN,NaN,NaN,NaN,142.025


In [24]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Pará - Desemprego,Pará - Produção de Cimento (t),Pará - value,Pará - IDH,Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-0.743008,-1.860097,-1.001954,-1.906833,-1.652592,-1.810463,-2.287151,-1.950096
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-0.748918,-1.858558,-1.004769,-1.880912,-1.635387,-1.758101,-2.245800,-1.920309
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-0.754829,-1.848873,-1.017098,-1.854991,-1.618182,-1.705740,-2.204450,-1.890522
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-0.760740,-1.850531,-1.033739,-1.829071,-1.600977,-1.653378,-2.163099,-1.860734
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-0.766650,-1.838250,-1.052007,-1.803150,-1.583772,-1.601017,-2.121748,-1.830947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.076246,-0.644020,1.421704,1.389768,1.316401,-0.980846,1.292832,1.315283
188,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.073514,-0.676398,1.447588,1.378488,1.314484,-0.973932,1.302757,1.318820
189,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.070782,-0.703221,1.485640,1.367208,1.312566,-0.967018,1.312682,1.322356
190,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.068050,-0.725198,1.528780,1.355927,1.310648,-0.960105,1.322607,1.325893


In [25]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      110.204
1       76.354
2       77.171
3       80.720
4       82.088
        ...   
235        NaN
236        NaN
237        NaN
238        NaN
239        NaN
Name: Pará - Consumo de Cimento (t), Length: 240, dtype: float64

In [26]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Pará - Desemprego,Pará - Produção de Cimento (t),Pará - value,Pará - IDH,Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado
0,2.723741,4.398348,2.052494,3.890153,-2.042341,-2.389042,3.122582,-0.743008,-1.860097,-1.001954,-1.906833,-1.652592,-1.810463,-2.287151,-1.950096
1,2.350880,4.222509,1.242280,3.551840,-2.014760,-2.352139,2.970356,-0.748918,-1.858558,-1.004769,-1.880912,-1.635387,-1.758101,-2.245800,-1.920309
2,2.123016,4.026019,2.936580,3.391423,-1.987179,-2.315236,2.869895,-0.754829,-1.848873,-1.017098,-1.854991,-1.618182,-1.705740,-2.204450,-1.890522
3,2.021477,3.811492,1.094024,3.135979,-1.959598,-2.278333,2.773628,-0.760740,-1.850531,-1.033739,-1.829071,-1.600977,-1.653378,-2.163099,-1.860734
4,1.887113,3.567576,2.351810,2.955412,-1.932017,-2.241431,2.977624,-0.766650,-1.838250,-1.052007,-1.803150,-1.583772,-1.601017,-2.121748,-1.830947
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,-0.214006,-0.607704,0.059280,-1.233012,1.031384,0.819304,-0.883659,1.444243,0.582557,1.197817,1.368533,1.186547,-1.094003,0.861099,1.054711
158,-0.434717,-0.620523,0.222389,-1.299304,1.042716,0.808136,-0.950771,1.421118,0.538378,1.208551,1.376125,1.197116,-1.099823,0.885880,1.071769
159,-0.524091,-0.631530,0.191929,-1.248662,1.054049,0.796969,-1.028465,1.397993,0.482165,1.218732,1.383716,1.207686,-1.105643,0.910662,1.088827
160,-0.614500,-0.640320,0.385687,-1.068274,1.065381,0.785801,-1.103668,1.374869,0.427013,1.224310,1.391308,1.218256,-1.111463,0.935444,1.105885


In [27]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      110.204
1       76.354
2       77.171
3       80.720
4       82.088
        ...   
157    116.872
158    131.959
159    116.171
160    132.242
161    135.615
Name: Pará - Consumo de Cimento (t), Length: 162, dtype: float64

In [28]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [29]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 15)

In [30]:
# input para treinamento
test_input = input_data.iloc[train_split - window_size:split_index + 1]
test_input

,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Pará - Desemprego,Pará - Produção de Cimento (t),Pará - value,Pará - IDH,Pará - PIB - Estadual,Pará - PIB - Construção Civil,Pará - PIB - Per Capita,Pará - PIB - Preços de Mercado
126,0.888984,-0.460555,-1.131704,-0.368821,0.651397,0.944085,0.045243,0.937142,1.114270,0.390386,0.491346,0.718639,0.747580,0.575976,0.685697
127,0.954254,-0.440372,-0.613483,-0.328087,0.664707,0.947319,0.061828,0.968970,1.082022,0.414807,0.506090,0.735578,0.698813,0.571078,0.692184
128,1.045217,-0.419247,-0.542919,-0.176031,0.678017,0.950553,0.046225,1.000798,1.052039,0.439827,0.520835,0.752516,0.650046,0.566181,0.698671
129,1.176395,-0.397019,-1.334517,-0.113037,0.691327,0.953786,0.032522,1.032625,1.020267,0.471477,0.535579,0.769455,0.601279,0.561283,0.705157
130,1.303259,-0.376532,-1.511762,-0.022703,0.704637,0.957020,0.042757,1.064453,0.982058,0.505972,0.550323,0.786394,0.552512,0.556385,0.711644
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-2.010387,-0.572934,-1.281958,0.589021,1.365911,0.389193,-1.749976,1.076246,-0.644020,1.421704,1.389768,1.316401,-0.980846,1.292832,1.315283
188,-1.870713,-0.588777,-1.358588,1.043728,1.376610,0.370392,-1.593005,1.073514,-0.676398,1.447588,1.378488,1.314484,-0.973932,1.302757,1.318820
189,-1.806230,-0.612606,-1.511565,1.387010,1.387308,0.351592,-1.351489,1.070782,-0.703221,1.485640,1.367208,1.312566,-0.967018,1.312682,1.322356
190,-1.727496,-0.640956,-1.421708,1.815728,1.398006,0.332791,-1.198492,1.068050,-0.725198,1.528780,1.355927,1.310648,-0.960105,1.322607,1.325893


In [31]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test.shape 

(30, 36, 15)

In [32]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [33]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(36, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(180,activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [34]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(50)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [35]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[3941262255, 1849367829, 3980720138, 4249900428, 3744345392, 4157486017, 137811426, 598631874, 1337220656, 2092927911, 1358260959, 571223763, 1308639691, 3084344773, 1993363318, 3171732780, 3968950972, 786672352, 139863129, 1129486062, 1132339643, 2863766753, 3000175947, 4164537423, 3631158148, 3729896349, 1309176394, 3580565787, 3140245167, 1766268013, 1131319265, 2375504004, 3034908312, 380499270, 348938963, 660702937, 3134492190, 3079520887, 2370358405, 470766755, 3875422938, 2353290412, 265249641, 3504881742, 1363170514, 1595341372, 3641315353, 3759156100, 3352169951, 4017786807]


Step: 0 ___________________________________________
val_loss: 80.40562438964844
winner_seed: 3941262255


Step: 1 ___________________________________________
val_loss: 102.61985778808594


Step: 2 ___________________________________________
val_loss: 85.76984405517578


Step: 3 ___________________________________________
val_loss: 121.24382781982422


Step: 4 ___________________________________________
v

In [36]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 131ms/step - loss: 19071.3027 - val_loss: 2067.0244
Epoch 2/10000
4/4 [==============================] - 0s 12ms/step - loss: 1449.2789 - val_loss: 1435.3295
Epoch 3/10000
4/4 [==============================] - 0s 11ms/step - loss: 1340.4197 - val_loss: 1322.9084
Epoch 4/10000
4/4 [==============================] - 0s 12ms/step - loss: 1342.9374 - val_loss: 1380.8959
Epoch 5/10000
4/4 [==============================] - 0s 12ms/step - loss: 1203.4325 - val_loss: 1317.7589
Epoch 6/10000
4/4 [==============================] - 0s 11ms/step - loss: 1201.9368 - val_loss: 1899.6708
Epoch 7/10000
4/4 [==============================] - 0s 12ms/step - loss: 1529.7675 - val_loss: 1422.8601
Epoch 8/10000
4/4 [==============================] - 0s 12ms/step - loss: 1197.1617 - val_loss: 1106.8225
Epoch 9/10000
4/4 [==============================] - 0s 12ms/step - loss: 952.4188 - val_loss: 912.0018
Epoch 10/10000
4/4 [==========================

Epoch 79/10000
4/4 [==============================] - 0s 11ms/step - loss: 203.4810 - val_loss: 240.0044
Epoch 80/10000
4/4 [==============================] - 0s 11ms/step - loss: 215.2816 - val_loss: 292.7723
Epoch 81/10000
4/4 [==============================] - 0s 11ms/step - loss: 210.7897 - val_loss: 234.4332
Epoch 82/10000
4/4 [==============================] - 0s 11ms/step - loss: 194.5005 - val_loss: 227.1756
Epoch 83/10000
4/4 [==============================] - 0s 12ms/step - loss: 193.8030 - val_loss: 258.8728
Epoch 84/10000
4/4 [==============================] - 0s 11ms/step - loss: 224.8504 - val_loss: 262.2050
Epoch 85/10000
4/4 [==============================] - 0s 10ms/step - loss: 232.6152 - val_loss: 233.0744
Epoch 86/10000
4/4 [==============================] - 0s 10ms/step - loss: 199.4109 - val_loss: 258.6548
Epoch 87/10000
4/4 [==============================] - 0s 9ms/step - loss: 200.9604 - val_loss: 332.3165
Epoch 88/10000
4/4 [==============================] - 0s

4/4 [==============================] - 0s 11ms/step - loss: 183.6985 - val_loss: 303.2031
Epoch 157/10000
4/4 [==============================] - 0s 13ms/step - loss: 193.6593 - val_loss: 227.4636
Epoch 158/10000
4/4 [==============================] - 0s 13ms/step - loss: 172.3219 - val_loss: 285.2460
Epoch 159/10000
4/4 [==============================] - 0s 14ms/step - loss: 155.9256 - val_loss: 206.8255
Epoch 160/10000
4/4 [==============================] - 0s 12ms/step - loss: 171.2125 - val_loss: 267.6153
Epoch 161/10000
4/4 [==============================] - 0s 11ms/step - loss: 187.7691 - val_loss: 303.6703
Epoch 162/10000
4/4 [==============================] - 0s 11ms/step - loss: 228.0854 - val_loss: 275.2227
Epoch 163/10000
4/4 [==============================] - 0s 11ms/step - loss: 188.3965 - val_loss: 237.5773
Epoch 164/10000
4/4 [==============================] - 0s 11ms/step - loss: 172.6216 - val_loss: 306.6260
Epoch 165/10000
4/4 [==============================] - 0s 11ms

4/4 [==============================] - 0s 11ms/step - loss: 169.4324 - val_loss: 247.1757
Epoch 234/10000
4/4 [==============================] - 0s 11ms/step - loss: 208.9752 - val_loss: 261.1355
Epoch 235/10000
4/4 [==============================] - 0s 12ms/step - loss: 166.3215 - val_loss: 257.2562
Epoch 236/10000
4/4 [==============================] - 0s 12ms/step - loss: 162.4669 - val_loss: 245.0350
Epoch 237/10000
4/4 [==============================] - 0s 13ms/step - loss: 164.2973 - val_loss: 285.0742
Epoch 238/10000
4/4 [==============================] - 0s 12ms/step - loss: 151.6550 - val_loss: 255.3242
Epoch 239/10000
4/4 [==============================] - 0s 13ms/step - loss: 167.2598 - val_loss: 258.6796
Epoch 240/10000
4/4 [==============================] - 0s 11ms/step - loss: 159.5751 - val_loss: 264.8795
Epoch 241/10000
4/4 [==============================] - 0s 12ms/step - loss: 157.9073 - val_loss: 251.3418
Epoch 242/10000
4/4 [==============================] - 0s 11ms

4/4 [==============================] - 0s 12ms/step - loss: 165.2202 - val_loss: 256.7634
Epoch 311/10000
4/4 [==============================] - 0s 11ms/step - loss: 145.6115 - val_loss: 297.0189
Epoch 312/10000
4/4 [==============================] - 0s 11ms/step - loss: 157.7258 - val_loss: 258.0673
Epoch 313/10000
4/4 [==============================] - 0s 11ms/step - loss: 176.8246 - val_loss: 238.8298
Epoch 314/10000
4/4 [==============================] - 0s 11ms/step - loss: 161.9356 - val_loss: 327.1406
Epoch 315/10000
4/4 [==============================] - 0s 11ms/step - loss: 186.6530 - val_loss: 247.9633
Epoch 316/10000
4/4 [==============================] - 0s 11ms/step - loss: 145.3915 - val_loss: 216.2898
Epoch 317/10000
4/4 [==============================] - 0s 11ms/step - loss: 170.9286 - val_loss: 238.4084
Epoch 318/10000
4/4 [==============================] - 0s 10ms/step - loss: 150.9798 - val_loss: 230.8582
Epoch 319/10000
4/4 [==============================] - 0s 10ms

...

4/4 [==============================] - 0s 12ms/step - loss: 47.1551 - val_loss: 154.0546
Epoch 1855/10000
4/4 [==============================] - 0s 11ms/step - loss: 45.1869 - val_loss: 169.4843
Epoch 1856/10000
4/4 [==============================] - 0s 11ms/step - loss: 44.3025 - val_loss: 142.4556
Epoch 1857/10000
4/4 [==============================] - 0s 11ms/step - loss: 46.6187 - val_loss: 147.0482
Epoch 1858/10000
4/4 [==============================] - 0s 12ms/step - loss: 47.6147 - val_loss: 158.1261
Epoch 1859/10000
4/4 [==============================] - 0s 12ms/step - loss: 43.9815 - val_loss: 150.3149
Epoch 1860/10000
4/4 [==============================] - 0s 11ms/step - loss: 41.6366 - val_loss: 150.4237
Epoch 1861/10000
4/4 [==============================] - 0s 11ms/step - loss: 38.6603 - val_loss: 171.1694
Epoch 1862/10000
4/4 [==============================] - 0s 12ms/step - loss: 50.3620 - val_loss: 136.3605
Epoch 1863/10000
4/4 [==============================] - 0s 11ms

4/4 [==============================] - 0s 11ms/step - loss: 29.6833 - val_loss: 155.9914
Epoch 1932/10000
4/4 [==============================] - 0s 11ms/step - loss: 30.4337 - val_loss: 194.9723
Epoch 1933/10000
4/4 [==============================] - 0s 11ms/step - loss: 34.0973 - val_loss: 147.2581
Epoch 1934/10000
4/4 [==============================] - 0s 11ms/step - loss: 34.3470 - val_loss: 153.7822
Epoch 1935/10000
4/4 [==============================] - 0s 11ms/step - loss: 30.2114 - val_loss: 128.9652
Epoch 1936/10000
4/4 [==============================] - 0s 11ms/step - loss: 39.5380 - val_loss: 133.9178
Epoch 1937/10000
4/4 [==============================] - 0s 11ms/step - loss: 44.4729 - val_loss: 133.1668
Epoch 1938/10000
4/4 [==============================] - 0s 12ms/step - loss: 41.2311 - val_loss: 165.2130
Epoch 1939/10000
4/4 [==============================] - 0s 11ms/step - loss: 34.3247 - val_loss: 150.3236
Epoch 1940/10000
4/4 [==============================] - 0s 12ms

4/4 [==============================] - 0s 11ms/step - loss: 26.9008 - val_loss: 153.7372
Epoch 2009/10000
4/4 [==============================] - 0s 11ms/step - loss: 30.8171 - val_loss: 161.3412
Epoch 2010/10000
4/4 [==============================] - 0s 12ms/step - loss: 26.7913 - val_loss: 160.4891
Epoch 2011/10000
4/4 [==============================] - 0s 11ms/step - loss: 39.0658 - val_loss: 151.6307
Epoch 2012/10000
4/4 [==============================] - 0s 11ms/step - loss: 33.6815 - val_loss: 162.7533
Epoch 2013/10000
4/4 [==============================] - 0s 12ms/step - loss: 31.1203 - val_loss: 171.9719
Epoch 2014/10000
4/4 [==============================] - 0s 11ms/step - loss: 53.6098 - val_loss: 141.4873
Epoch 2015/10000
4/4 [==============================] - 0s 11ms/step - loss: 34.3613 - val_loss: 149.3155
Epoch 2016/10000
4/4 [==============================] - 0s 11ms/step - loss: 24.1414 - val_loss: 167.9055
Epoch 2017/10000
4/4 [==============================] - 0s 12ms

4/4 [==============================] - 0s 11ms/step - loss: 30.9281 - val_loss: 146.5758
Epoch 2086/10000
4/4 [==============================] - 0s 10ms/step - loss: 24.6035 - val_loss: 134.0065
Epoch 2087/10000
4/4 [==============================] - 0s 10ms/step - loss: 33.7971 - val_loss: 151.0649
Epoch 2088/10000
4/4 [==============================] - 0s 10ms/step - loss: 24.9580 - val_loss: 179.2426
Epoch 2089/10000
4/4 [==============================] - 0s 11ms/step - loss: 26.4409 - val_loss: 149.0855
Epoch 2090/10000
4/4 [==============================] - 0s 11ms/step - loss: 23.3908 - val_loss: 202.1353
Epoch 2091/10000
4/4 [==============================] - 0s 11ms/step - loss: 24.8141 - val_loss: 139.5681
Epoch 2092/10000
4/4 [==============================] - 0s 12ms/step - loss: 43.3205 - val_loss: 125.4241
Epoch 2093/10000
4/4 [==============================] - 0s 12ms/step - loss: 16.9405 - val_loss: 184.1655
Epoch 2094/10000
4/4 [==============================] - 0s 11ms

4/4 [==============================] - 0s 11ms/step - loss: 26.8334 - val_loss: 172.8247
Epoch 2163/10000
4/4 [==============================] - 0s 12ms/step - loss: 19.3361 - val_loss: 225.9352
Epoch 2164/10000
4/4 [==============================] - 0s 11ms/step - loss: 18.1272 - val_loss: 170.4955
Epoch 2165/10000
4/4 [==============================] - 0s 12ms/step - loss: 19.6948 - val_loss: 196.9733
Epoch 2166/10000
4/4 [==============================] - 0s 12ms/step - loss: 19.1343 - val_loss: 171.0822
Epoch 2167/10000
4/4 [==============================] - 0s 11ms/step - loss: 21.9271 - val_loss: 187.9558
Epoch 2168/10000
4/4 [==============================] - 0s 11ms/step - loss: 24.6973 - val_loss: 171.6452
Epoch 2169/10000
4/4 [==============================] - 0s 11ms/step - loss: 24.9818 - val_loss: 192.5646
Epoch 2170/10000
4/4 [==============================] - 0s 11ms/step - loss: 16.9661 - val_loss: 172.6503
Epoch 2171/10000
4/4 [==============================] - 0s 12ms

In [37]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [38]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 15ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,126.420586,126.755165,126.864281,126.658058,126.714394,126.981148,127.359138,127.490952,127.000633,126.930977,126.650856,125.859306,124.458916,123.604683,122.744514,121.76709,121.804085,120.966011,119.670578,118.396553,115.241745,113.510979,113.89045,112.304192,112.712067,120.252472,118.991341,121.011391,121.366982,122.23333
Target,136.117,136.475,125.3,132.627,135.172,131.376,111.175,80.97,92.173,86.782,85.15,135.094,132.351,133.744,128.527,147.51,138.511,103.294,110.704,91.638,90.744,82.108,97.49,96.803,118.687,109.761,120.188,131.62,125.374,115.811
Error,9.696419,9.719841,1.564278,5.968941,8.457603,4.394859,16.184135,46.52095,34.827637,40.148979,41.500854,9.234688,7.892082,10.13932,5.782478,25.742905,16.706917,17.672012,8.966576,26.758553,24.497742,31.402977,16.400452,15.50119,5.97493,10.49147,1.196663,10.608604,4.007019,6.422333


In [39]:
display(mae)
display(mape)

15.812778

0.15788847

In [40]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input.iloc[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [41]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             test_input, 
                                             test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

NameError: name 'test_target' is not defined